# **Avance 6**

## *Análisis fundamental*

Este analisis incluye los siguientes componentes:
* Análisis Top-Down (análisis macroeconomico)
* Análisis Bottom-Up (cálculo de indicadores como PER, Capitalización bursátil, Q de Tobin, Valor Intrínseco, Yield anual, ROA, ROE)
* Benchmark
* Calculo múltiplos 
* Valor por descuento de dividendos

## **Análisis Top-Down**

Se realiza un análisis de los factores que pueden afectar el valor de la compañia, estos se pueden encontrar en noticias o estados financieros y pueden ser a nivel global, del sector o industria o cía.

## **Análisis bottom-up**

En este análisis se utilizan indicadores fundamentales y valoración de acciones para comprender el estado de la acción

In [13]:
import pandas as pd
from pandas_datareader import data as  pdr
from bs4 import BeautifulSoup
import urllib.request as ur
import yfinance as yf

### Datos a trabajar

Sacamos la información que nos interesa y complemtamos los valores en el DataFrame creado para resumir los valores

In [14]:
#creamos un dataframe con los datos e informacion financiera
info_financiera = pd.DataFrame()
opciones = ['Precio Acción', 'Utilidades Netas', 'Acciones en Circulación', 'Patrimonio', 'Activos', 'Dividendo Anual', 'Utilidad operativa']
info_financiera['Datos'] = opciones
info_financiera = info_financiera.set_index('Datos')
info_financiera['Valores'] = None
info_financiera

,Valores
Datos,
Precio Acción,None
Utilidades Netas,None
Acciones en Circulación,None
Patrimonio,None
Activos,None
Dividendo Anual,None
Utilidad operativa,None


In [15]:
# Enter a stock symbol
index= 'FDX'
# URL link 
url_is = 'https://finance.yahoo.com/quote/' + index + '/financials?p=' + index
url_bs = 'https://finance.yahoo.com/quote/' + index + '/balance-sheet?p=' + index
url_cf = 'https://finance.yahoo.com/quote/' + index + '/cash-flow?p=' + index
url_summary = 'https://finance.yahoo.com/quote/' + index + '?p=' + index
url_statistics = 'https://finance.yahoo.com/quote/' + index + '/key-statistics?p=' + index

In [16]:
def findAndReplace(lista, itemAReemplazar, nuevoValor): 
    for index, item in enumerate(lista):
        if item == itemAReemplazar:
            lista[index] = nuevoValor

#### Income statement

In [17]:
import requests
read_data = requests.get(url_is,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})

In [18]:
soup_is= BeautifulSoup(read_data.content,"lxml")

divTag = soup_is.find_all("div", {"class": {"D(tbc)", "Ta(c)"}})

ls= []
ls.append("Breakdown")

for tag in divTag:
  tag_value =  tag.get('value')
  ls.append(tag.text)

for i in ls:
  if i == '-': findAndReplace(ls, '-', '0')

contador = 0
for i in ls:
  if i=='ttm':
    del ls[1:contador]
  else: contador += 1

new_ls = list(filter(None,ls))
is_data = list(zip(*[iter(new_ls)]*6))


In [19]:
Income_st = pd.DataFrame(is_data[0:])
Income_st

,0,1,2,3,4,5
0,Breakdown,ttm,5/31/2022,5/31/2021,5/31/2020,5/31/2019
1,Total Revenue,"94,091,000","93,512,000","83,959,000","69,217,000","69,693,000"
2,Cost of Revenue,"74,104,000","73,345,000","66,005,000","55,873,000","54,866,000"
3,Gross Profit,"19,987,000","20,167,000","17,954,000","13,344,000","14,827,000"
4,Operating Expense,"14,129,000","13,644,000","11,981,000","10,492,000","10,041,000"
5,Operating Income,"5,858,000","6,523,000","5,973,000","2,852,000","4,786,000"
6,Net Non Operating Interest Income Expense,"-590,000","-636,000","-741,000","-617,000","-529,000"
7,Other Income Expense,"-996,000","-991,000","1,442,000","-566,000","-3,602,000"
8,Pretax Income,"4,272,000","4,896,000","6,674,000","1,669,000","655,000"
9,Tax Provision,"939,000","1,070,000","1,443,000","383,000","115,000"


In [20]:
fila = 34 #numero de la fila que hay que eliminar
Income_st = Income_st.drop(fila,axis=0)
Income_st.columns = Income_st.iloc[0] #Nombramos las columnas con la primera fila del dataframe
Income_st.drop(Income_st.index[0],inplace=True) #eliminamos la primera fila
Income_st = Income_st.set_index('Breakdown') #nombramos los indices del Dataframe con los valores de la columna breakdown
Income_st

,ttm,5/31/2022,5/31/2021,5/31/2020,5/31/2019
Breakdown,,,,,
Total Revenue,"94,091,000","93,512,000","83,959,000","69,217,000","69,693,000"
Cost of Revenue,"74,104,000","73,345,000","66,005,000","55,873,000","54,866,000"
Gross Profit,"19,987,000","20,167,000","17,954,000","13,344,000","14,827,000"
Operating Expense,"14,129,000","13,644,000","11,981,000","10,492,000","10,041,000"
Operating Income,"5,858,000","6,523,000","5,973,000","2,852,000","4,786,000"
Net Non Operating Interest Income Expense,"-590,000","-636,000","-741,000","-617,000","-529,000"
Other Income Expense,"-996,000","-991,000","1,442,000","-566,000","-3,602,000"
Pretax Income,"4,272,000","4,896,000","6,674,000","1,669,000","655,000"
Tax Provision,"939,000","1,070,000","1,443,000","383,000","115,000"


#### Balance Sheet

In [21]:
import requests
read_data = requests.get(url_bs,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})


In [22]:
soup_is= BeautifulSoup(read_data.content,"lxml")

divTag = soup_is.find_all("div", {"class": {"D(tbc)", "Ta(c)"}})

ls= []
ls.append("Breakdown")

for tag in divTag:
  tag_value =  tag.get('value')
  ls.append(tag.text)

for i in ls:
  if i == '-': findAndReplace(ls, '-', '0')

contador = 0
for i in ls:
  if i=='5/31/2022':
    del ls[1:contador]
  else: contador += 1

new_ls = list(filter(None,ls))
is_data = list(zip(*[iter(new_ls)]*5))

In [23]:
Balance_sheet = pd.DataFrame(is_data[0:])
Balance_sheet

,0,1,2,3,4
0,Breakdown,5/31/2022,5/31/2021,5/31/2020,5/31/2019
1,Total Assets,"85,994,000","82,777,000","73,537,000","54,403,000"
2,Total Liabilities Net Minority Interest,"61,055,000","58,609,000","55,242,000","36,646,000"
3,Total Equity Gross Minority Interest,"24,939,000","24,168,000","18,295,000","17,757,000"
4,Total Capitalization,"44,685,000","44,472,000","39,813,000","34,374,000"
5,Common Stock Equity,"24,939,000","24,168,000","18,295,000","17,757,000"
6,Capital Lease Obligations,"17,398,000","16,108,000","14,603,000",0
7,Net Tangible Assets,"18,094,000","16,824,000","11,601,000","10,457,000"
8,Working Capital,"6,091,000","6,920,000","6,039,000","4,073,000"
9,Invested Capital,"44,735,000","44,522,000","39,813,000","35,338,000"


In [24]:
fila = 16 #numero de la fila que hay que eliminar
Balance_sheet = Balance_sheet.drop(fila,axis=0)
Balance_sheet.columns = Balance_sheet.iloc[0] #Nombramos las columnas con la primera fila del dataframe
Balance_sheet.drop(Balance_sheet.index[0],inplace=True) #eliminamos la primera fila
Balance_sheet = Balance_sheet.set_index('Breakdown')#nombramos los indices del Dataframe con los valores de la columna breakdown
Balance_sheet

,5/31/2022,5/31/2021,5/31/2020,5/31/2019
Breakdown,,,,
Total Assets,"85,994,000","82,777,000","73,537,000","54,403,000"
Total Liabilities Net Minority Interest,"61,055,000","58,609,000","55,242,000","36,646,000"
Total Equity Gross Minority Interest,"24,939,000","24,168,000","18,295,000","17,757,000"
Total Capitalization,"44,685,000","44,472,000","39,813,000","34,374,000"
Common Stock Equity,"24,939,000","24,168,000","18,295,000","17,757,000"
Capital Lease Obligations,"17,398,000","16,108,000","14,603,000",0
Net Tangible Assets,"18,094,000","16,824,000","11,601,000","10,457,000"
Working Capital,"6,091,000","6,920,000","6,039,000","4,073,000"
Invested Capital,"44,735,000","44,522,000","39,813,000","35,338,000"


#### Datos de resumen de la acción

In [25]:
read_data = requests.get(url_summary,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})

In [26]:
soup_is= BeautifulSoup(read_data.content,"lxml")

divTag = soup_is.find_all("td", {"class": {"C($primaryColor)","W(51%)","Ta(end)","Fw(600)","Lh(14px)"}})

ls= []
ls.append("Name")
ls.append("Value")

for tag in divTag:
  tag_value =  tag.get('value')
  ls.append(tag.text)

new_ls = list(filter(None,ls))
is_data = list(zip(*[iter(new_ls)]*2))

summary = pd.DataFrame(is_data[0:])
summary.columns = summary.iloc[0] #Nombramos las columnas con la primera fila del dataframe
summary.drop(summary.index[0],inplace=True) #eliminamos la primera fila
summary = summary.set_index('Name')#nombramos los indices del Dataframe con los valores de la columna breakdown
summary

,Value
Name,
Previous Close,204.05
Open,226.05
Bid,219.23 x 1000
Ask,219.33 x 800
Day's Range,218.54 - 226.31
52 Week Range,141.92 - 248.76
Volume,"5,027,872"
Avg. Volume,"2,101,448"
Market Cap,55.537B


#### Múltiplos

In [27]:
import requests
read_data = requests.get(url_statistics,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})

In [28]:
soup_is= BeautifulSoup(read_data.content,"lxml")

divTag = soup_is.find_all("td", {"class": {"Fw(500)","Ta(end)","Pstart(10px)","Miw(60px)","Pos(st)","Start(0)"}})

ls= []
ls.append("Name")
ls.append("Value")

for tag in divTag:
  tag_value =  tag.get('value')
  ls.append(tag.text)

new_ls = list(filter(None,ls))
is_data = list(zip(*[iter(new_ls)]*2))

statistics = pd.DataFrame(is_data[0:])
statistics.columns = statistics.iloc[0] #Nombramos las columnas con la primera fila del dataframe
statistics.drop(statistics.index[0],inplace=True) #eliminamos la primera fila
statistics = statistics.set_index('Name')#nombramos los indices del Dataframe con los valores de la columna breakdown
statistics


,Value
Name,
Market Cap (intraday),52.07B
Enterprise Value,85.46B
Trailing P/E,16.16
Forward P/E,11.81
PEG Ratio (5 yr expected),1.29
Price/Sales (ttm),0.57
Price/Book (mrq),2.16
Enterprise Value/Revenue,0.91
Enterprise Value/EBITDA,9.50


#### Dividendos

In [29]:
pip install yahoo-fin

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
from yahoo_fin import stock_info as si
aux_div = si.get_dividends(index)
aux_div = aux_div.drop(['ticker'], axis = 1)
aux_div = aux_div.resample('A').sum()
aux_div

,dividend
2002-12-31,0.15
2003-12-31,0.21
2004-12-31,0.27
2005-12-31,0.31
2006-12-31,0.35
2007-12-31,0.39
2008-12-31,0.43
2009-12-31,0.44
2010-12-31,0.47
2011-12-31,0.51


#### Completamos el Dataframe

In [31]:
info_financiera['Valores']['Precio Acción'] = summary['Value']['Previous Close']
info_financiera['Valores']['Utilidades Netas'] = Income_st['5/31/2022']['Net Income Common Stockholders']
info_financiera['Valores']['Acciones en Circulación'] = Balance_sheet['5/31/2022']['Share Issued']
info_financiera['Valores']['Patrimonio'] = Balance_sheet['5/31/2022']['Total Equity Gross Minority Interest']
info_financiera['Valores']['Activos'] = Balance_sheet['5/31/2022']['Total Assets']
info_financiera['Valores']['Dividendo Anual'] = aux_div['dividend']['2022-12-31']
info_financiera['Valores']['Utilidad operativa'] = Income_st['5/31/2022']['Operating Income']
info_financiera

,Valores
Datos,
Precio Acción,204.05
Utilidades Netas,"3,819,000"
Acciones en Circulación,"318,000"
Patrimonio,"24,939,000"
Activos,"85,994,000"
Dividendo Anual,4.2
Utilidad operativa,"6,523,000"


In [32]:
info_financiera['Valores']['Utilidades Netas'] = info_financiera['Valores']['Utilidades Netas'].replace(",", "")
info_financiera['Valores']['Acciones en Circulación'] = info_financiera['Valores']['Acciones en Circulación'].replace(",", "")
info_financiera['Valores']['Patrimonio'] = info_financiera['Valores']['Patrimonio'].replace(",", "")
info_financiera['Valores']['Activos'] = info_financiera['Valores']['Activos'].replace(",", "")
info_financiera['Valores']['Utilidad operativa'] = info_financiera['Valores']['Utilidad operativa'].replace(",", "")
info_financiera


,Valores
Datos,
Precio Acción,204.05
Utilidades Netas,3819000
Acciones en Circulación,318000
Patrimonio,24939000
Activos,85994000
Dividendo Anual,4.2
Utilidad operativa,6523000


In [33]:
#los valores del income statement y del balance sheet estan en miles
#para que queden en unidades, multiplicamos por 1000
info_financiera['Valores']['Utilidades Netas'] = int(info_financiera['Valores']['Utilidades Netas'])*1000
info_financiera['Valores']['Acciones en Circulación'] = int(info_financiera['Valores']['Acciones en Circulación'])*1000
info_financiera['Valores']['Patrimonio'] = int(info_financiera['Valores']['Patrimonio'])*1000
info_financiera['Valores']['Activos'] = int(info_financiera['Valores']['Activos'])*1000
info_financiera['Valores']['Utilidad operativa'] = int(info_financiera['Valores']['Utilidad operativa'])*1000
info_financiera

,Valores
Datos,
Precio Acción,204.05
Utilidades Netas,3819000000
Acciones en Circulación,318000000
Patrimonio,24939000000
Activos,85994000000
Dividendo Anual,4.2
Utilidad operativa,6523000000


### Indicadores

Se crea un dataframe con la informacion de los indicadores, realizando comparaciones entre los valores presentados en tablas por la acción y el valor calculado

In [34]:
#creamos un dataframe con la informacion que vamos a utilizar y al final comparar
indicadores = pd.DataFrame()
opciones = ['UPA', 'PER', 'Capitalización Bursátil', 'Q de Tobin', 'Valor Intrínseco', 'Yield Anual', 'ROA', 'ROE']
indicadores["Indicador"] = opciones
indicadores = indicadores.set_index('Indicador')
indicadores["Valor Tablas"] = None
indicadores["Valor Calculado"] = None
indicadores

,Valor Tablas,Valor Calculado
Indicador,,
UPA,None,None
PER,None,None
Capitalización Bursátil,None,None
Q de Tobin,None,None
Valor Intrínseco,None,None
Yield Anual,None,None
ROA,None,None
ROE,None,None


#### *UPA*

Tablas = Basic EPC (income statement)

Calculado = utilidades netas / acciones en circulación

Interpretación = número de veces que la utilidad está comprendido en el precio de la acción. Lo que le toca a cada acción de la utilidad de la compañía

In [35]:
indicadores['Valor Tablas']['UPA'] = Income_st['5/31/2022']['Basic EPS']
indicadores['Valor Calculado']['UPA'] = int(info_financiera['Valores']['Utilidades Netas']) / int(info_financiera['Valores']['Acciones en Circulación'])
indicadores

,Valor Tablas,Valor Calculado
Indicador,,
UPA,14.38,12.009434
PER,None,None
Capitalización Bursátil,None,None
Q de Tobin,None,None
Valor Intrínseco,None,None
Yield Anual,None,None
ROA,None,None
ROE,None,None


#### *Método PER*

Tablas = PER (summary)

Calculado = precio accion / UPA

Interpretación = cuantas veces la utilidad por acción cabe en el precio, cuanto tiempo necesito para pagar la acción

In [36]:
indicadores['Valor Tablas']['PER'] = summary['Value']['PE Ratio (TTM)'] #tener en cuenta que este es trimestral, no anual
indicadores['Valor Calculado']['PER'] = float(info_financiera['Valores']['Precio Acción']) / float(indicadores['Valor Calculado']['UPA'])
indicadores

,Valor Tablas,Valor Calculado
Indicador,,
UPA,14.38,12.009434
PER,16.49,16.990809
Capitalización Bursátil,None,None
Q de Tobin,None,None
Valor Intrínseco,None,None
Yield Anual,None,None
ROA,None,None
ROE,None,None


#### *Capitalización Bursátil*

Tablas = market cap (summary)

Calculado = precio accion * # de acciones en circulación

Interpretación = muestra el valor a precios de mercados de todas las acciones de circulacion. Muestra cuánto vale la empresa en el mercado

In [37]:
indicadores['Valor Tablas']['Capitalización Bursátil'] = summary['Value']['Market Cap']
indicadores['Valor Calculado']['Capitalización Bursátil'] = float(info_financiera['Valores']['Precio Acción']) * int(info_financiera['Valores']['Acciones en Circulación'])
indicadores


,Valor Tablas,Valor Calculado
Indicador,,
UPA,14.38,12.009434
PER,16.49,16.990809
Capitalización Bursátil,55.537B,64887900000.0
Q de Tobin,None,None
Valor Intrínseco,None,None
Yield Anual,None,None
ROA,None,None
ROE,None,None


#### *Q de Tobin*

Tablas = no sale

Calculado = precio de mercado / valor intrínseco = capitalización busrátil / valor patrimonio en libros

Interpretación = 
* Si Q > 1, la acción esta sobrevalorada (el mercado ve más de lo que realmente es o presenta una accion en sus cifras financieras, se espera que a mediano o corto plazo baje)
* Si Q < 1, la acción esta subvalorada (la acción no está generando confianza en el mercado, vale menos de lo que vale nominalmente)

In [38]:
indicadores['Valor Tablas']['Q de Tobin'] = 0
indicadores['Valor Calculado']['Q de Tobin'] = float(indicadores['Valor Calculado']['Capitalización Bursátil']) / int(info_financiera['Valores']['Patrimonio'])
indicadores

,Valor Tablas,Valor Calculado
Indicador,,
UPA,14.38,12.009434
PER,16.49,16.990809
Capitalización Bursátil,55.537B,64887900000.0
Q de Tobin,0,2.601865
Valor Intrínseco,None,None
Yield Anual,None,None
ROA,None,None
ROE,None,None


#### *Valor intrínseco*

Tablas = no sale

Calculado = patrimonio en libros / # de acciones en circulación

Interpretación = trata de reflejar la realidad económica de la acción, describe el valor percibido o real de un activo

In [39]:
indicadores['Valor Tablas']['Valor Intrínseco'] = 0
indicadores['Valor Calculado']['Valor Intrínseco'] = int(info_financiera['Valores']['Patrimonio']) / int(info_financiera['Valores']['Acciones en Circulación'])
indicadores

,Valor Tablas,Valor Calculado
Indicador,,
UPA,14.38,12.009434
PER,16.49,16.990809
Capitalización Bursátil,55.537B,64887900000.0
Q de Tobin,0,2.601865
Valor Intrínseco,0,78.424528
Yield Anual,None,None
ROA,None,None
ROE,None,None


#### *Yield*

Tablas = Forward Annual Dividend Yield

Calculado = dividendos del último año (anual) / precio acción

Interpretación = rendimiento generado por la empresa a los accionistas

In [41]:
indicadores['Valor Tablas']['Yield Anual'] = statistics['Value']['Forward Annual Dividend Yield 4']
indicadores['Valor Calculado']['Yield Anual'] = float(info_financiera['Valores']['Dividendo Anual']) / float(info_financiera['Valores']['Precio Acción'])
indicadores

,Valor Tablas,Valor Calculado
Indicador,,
UPA,14.38,12.009434
PER,16.49,16.990809
Capitalización Bursátil,55.537B,64887900000.0
Q de Tobin,0,2.601865
Valor Intrínseco,0,78.424528
Yield Anual,2.36%,0.020583
ROA,None,None
ROE,None,None


#### *ROA*

Tablas = Return on Assets (ttm -> trimestral)

Calculado = utilidad operativa / activos

Interpretación = medida financiera que se utiliza para evaluar la eficiencia de una empresa para generar ganancias a partir de sus activos. Esto indica cuánto beneficio se está generando por cada unidad de activos que posee la empresa

In [42]:
indicadores['Valor Tablas']['ROA'] = statistics['Value']['Return on Assets (ttm)']
indicadores['Valor Calculado']['ROA'] = int(info_financiera['Valores']['Utilidad operativa']) / int(info_financiera['Valores']['Activos'])
indicadores

,Valor Tablas,Valor Calculado
Indicador,,
UPA,14.38,12.009434
PER,16.49,16.990809
Capitalización Bursátil,55.537B,64887900000.0
Q de Tobin,0,2.601865
Valor Intrínseco,0,78.424528
Yield Anual,2.36%,0.020583
ROA,3.68%,0.075854
ROE,None,None


#### *ROE*

Tablas = Return on Equity (ttm -> trimestral)

Calculado = utilidad neta / patrimonio

Interpretación = medida financiera que se utiliza para evaluar la rentabilidad de una empresa en relación con la inversión de sus accionistas. Esto indica cuánto beneficio se está generando por cada unidad de inversión de los accionistas

In [43]:
indicadores['Valor Tablas']['ROE'] = statistics['Value']['Return on Equity (ttm)']
indicadores['Valor Calculado']['ROE'] = int(info_financiera['Valores']['Utilidades Netas']) / info_financiera['Valores']['Patrimonio']
indicadores

,Valor Tablas,Valor Calculado
Indicador,,
UPA,14.38,12.009434
PER,16.49,16.990809
Capitalización Bursátil,55.537B,64887900000.0
Q de Tobin,0,2.601865
Valor Intrínseco,0,78.424528
Yield Anual,2.36%,0.020583
ROA,3.68%,0.075854
ROE,12.15%,0.153134


## **Benchmarking**

En este análisis se trata de seleccionar empresas comparables de la compañía objeto de estudio e investigar los diferentes indicadores fundamentales para estas y estimar un promedio para cada indicador. Comparar los resultados de la compañía con dicho promedio y realizar conclusiones

In [45]:
#datos a trabajar
tabla_benchmark = pd.DataFrame()
tabla_benchmark['Acronimo'] = None
tabla_benchmark['PER'] = None
tabla_benchmark['Q de Tobin'] = None